In [2]:
import pandas as pd

train=pd.read_csv('../titanic/train.csv')
test=pd.read_csv('../titanic/test.csv')
train.shape # (891,12)
test.shape # (418,11)
train.head()

#성별, 선실등급별 생존율

train.groupby(['Sex','Pclass'])['Survived'].mean()
train.pivot_table(index=['Sex','Pclass'], values=['Survived']) #api문서에 가면 인수규칙이 나와있다.

#모델링
test['Survived'] = test.Sex=='female'
test.head()

test['Survived'].value_counts()
submission = test[['PassengerId','Survived']].copy()
submission.head()

submission['Survived'] = submission['Survived'].astype(int)
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [3]:
train.sample(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
109,110,1,3,"Moran, Miss. Bertha",female,NaN,1,0,371110,24.1500,NaN,Q
503,504,0,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.5875,NaN,S
488,489,0,3,"Somerton, Mr. Francis William",male,30.0,0,0,A.5. 18509,8.0500,NaN,S
780,781,1,3,"Ayoub, Miss. Banoura",female,13.0,0,0,2687,7.2292,NaN,C
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,A23,S
373,374,0,1,"Ringhini, Mr. Sante",male,22.0,0,0,PC 17760,135.6333,NaN,C
722,723,0,2,"Gillespie, Mr. William Henry",male,34.0,0,0,12233,13.0000,NaN,S
554,555,1,3,"Ohman, Miss. Velin",female,22.0,0,0,347085,7.7750,NaN,S
64,65,0,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C
699,700,0,3,"Humblen, Mr. Adolf Mathias Nicolai Olsen",male,42.0,0,0,348121,7.6500,F G63,S


In [71]:
sample = train.groupby(['Sex','Age'])['Survived'].mean()
sample.male.head(50) # 남자애기 6살이하 많이 살았음
sample2 = pd.DataFrame(train.groupby(['Sex','Pclass','Age'])['Survived'].mean())
sample2.tail(50) #남자 3등실 거의다 사망..ㅠ
# sample2 = sample2.stack()
sample2 = sample2.reset_index() ####!!!
f1 = sample2['Sex']=='male'
f0 = sample2['Sex']=='female'
f2 = sample2['Pclass']==2
f4 = sample2['Age'] <= 8
# sample2[f1][f2][f4] #남자 2등실 8살이하 생존 / 남자 1등실 60세 이하 생존
# sample2[f0][f2] #여자 3등실 40세 이상 사망

In [103]:
#모델링
# test['Survived'] = test.Sex=='female'
# test.head()

# test['Survived'].value_counts()
# submission = test[['PassengerId','Survived']].copy()
# submission.head()

# submission['Survived'] = submission['Survived'].astype(int)
# submission.head()

#사망자
d = (test.Sex=='female') & (test.Pclass==3) & (test.Age > 40)

#생존자
a = (test.Sex=='female')
b = (test.Sex=='male') & (test.Pclass==2) & (test.Age <= 8)
c = (test.Sex=='male') & (test.Pclass==1) & (test.Age <= 60)
#filter = (a | b | c)

test['Survived'] = a|b|c
test.groupby(['Sex',]).head(50)

# submission = test[['PassengerId','Survived']].copy()
# submission.head()

# submission['Survived'] = submission['Survived'].astype(int)
# submission.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,False
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,True
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,False
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,False
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,True
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S,False
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,True
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S,False
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,True
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S,False


In [102]:
submission.to_csv("submission2.csv", index = False)

In [ ]:
#submission.to_csv("submission.csv", index=False)